In [13]:
import pandas as pd
summary=pd.read_csv('./Data/Contest1245/standings_statistics.csv',sep='\t')
summary=summary[summary['ProblemC_language'].str.contains('C+')]
summary.head()

,ParticipantID,Rank,Country,ProblemA_id,ProblemA_memory,ProblemA_time,ProblemA_language,ProblemB_id,ProblemB_memory,ProblemB_time,ProblemB_language,ProblemC_id,ProblemC_memory,ProblemC_time,ProblemC_language
0,29614498,1,Japan,63995951,8,30,GNU C++17,63997617,8,46,GNU C++17,63999164,504,46,GNU C++17
1,29612723,2,-,63995957,4,31,GNU C++17,63998088,8,15,GNU C++17,64000498,1064,30,GNU C++17
2,29613216,3,-,63997656,4,31,GNU C++11,64001465,8,31,GNU C++11,64005851,500,31,GNU C++11
3,29565091,4,-,63996018,4,31,GNU C++11,63998805,1960,15,GNU C++11,64003393,9792,31,GNU C++11
4,29617008,5,China,63998793,4,30,GNU C++11,64003870,4,15,GNU C++11,64010042,500,46,GNU C++11


In [11]:
summary.shape

(957, 15)

In [1]:
#collect all code from standings folders into one folder
import os
from shutil import copyfile

for outer_d in os.listdir('./Data'):
    outer_d=os.path.join('./Data',outer_d)
    if os.path.isdir(outer_d):
        #create dir for all codes
        copy_dir=os.path.join(outer_d,"all")
        if not os.path.exists(copy_dir):
            os.makedirs(copy_dir) 
        for d in os.listdir(outer_d):
            d=os.path.join(outer_d,d)
            if os.path.isdir(d) and d!=copy_dir:
                for f in os.listdir(d):
                    file_name=f
                    f=os.path.join(d,f)
                    if os.path.isfile(f):
                        copyfile(f,os.path.join(copy_dir,file_name))

In [6]:
#generate xml for all files using srcml
for outer_d in os.listdir('./Data'):
    outer_d=os.path.join('./Data',outer_d) #Contest1,2,3,...
    if os.path.isdir(outer_d):
        #create dir for xml 
        xml_dir=os.path.join(outer_d,"all_xml")
        if not os.path.exists(xml_dir):
            os.makedirs(xml_dir) 
        for f in os.listdir(os.path.join(outer_d,'all')):
            fname=f
            f=os.path.join(os.path.join(outer_d,'all'),f)
            if os.path.isfile(f):
                cmd="srcml %s -o %s"%(f,os.path.join(xml_dir,fname.replace('.cpp','.xml')))
                os.system(cmd)

In [14]:
import subprocess
proc=subprocess.Popen('srcml --xpath "string(//src:function/src:type)" example.xml', stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
print("program output:", out)

program output: b'int\n'


In [71]:
import subprocess
from collections import defaultdict
import pandas as pd
import xml.etree.ElementTree as ET
import os

def get_leaf_tags(root,result=""):
    if len(root)==0:
        if root.text is not None:
            result=result+" "+root.text
        return result
    else:
        for child in root:
            result=get_leaf_tags(child,result)
        return result

def get_pointer_declarations(filename):
    x=lambda filename: '''srcml --xpath "//src:decl[src:type/src:modifier[.='*']]" %s '''%(filename)
    output_xml=call_shell(x,filename)
    root=ET.fromstring(output_xml)
    pointer_decl=[]
    for unit in root:
        pointer_decl.append(get_leaf_tags(unit))
    return pointer_decl

def get_reference_declarations(filename):
    x=lambda filename: '''srcml --xpath "//src:decl[src:type/src:modifier[.='&']]" %s '''%(filename)
    output_xml=call_shell(x,filename)
    root=ET.fromstring(output_xml)
    reference_decl=[]
    for unit in root:
        reference_decl.append(get_leaf_tags(unit))
    return reference_decl

def get_array_declarations(filename):
    x=lambda filename: '''srcml --xpath "//src:decl[src:name/src:index]" %s '''%(filename)
    output_xml=call_shell(x,filename)
    root=ET.fromstring(output_xml)
    array_decl=[]
    for unit in root:
        array_decl.append(get_leaf_tags(unit))
    return array_decl

def get_struct_def(filename):
    '''does not work, bad parsing by srcML'''
    x=lambda filename: '''srcml --xpath "//src:struct" %s '''%(filename)
    output_xml=call_shell(x,filename)
    root=ET.fromstring(output_xml)
    struct_decl=[]
    for unit in root:
        struct_decl.append(get_leaf_tags(unit))
    return struct_decl

def get_function_calls(filename):
    x=lambda filename: '''srcml --xpath "//src:call" %s '''%(filename)
    output_xml=call_shell(x,filename)
    root=ET.fromstring(output_xml)
    fn_calls=[]
    for unit in root:
        fn_calls.append(get_leaf_tags(unit))
    return fn_calls

def get_function_names(root):
    fn_names=[]
    for child in root:
        if child.tag.endswith('function'):
            for grandchild in child:
                if grandchild.tag.endswith('name'):
                    fn_names.append(grandchild.text)
    return fn_names

def get_codelines(filepath):
    

def call_shell(f,filepath):
    cmd=f(filepath)
    proc=subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    (out, err) = proc.communicate()
    return out

def get_features_for_file(filepath):
    features=[]
    feature_generators={'Function_Return_type':(lambda filepath: ('srcml --xpath "string(//src:function/src:type)" %s')%(filepath),"XPATH"), 
                        'Function_Params':(lambda filepath: ('srcml --xpath "string(//src:function/src:parameter_list/src:parameter)" %s'%(filepath)),"XPATH"),
                        'Function_Name': (lambda root: get_function_names(root),"FUNCTION"),
                        'Typedefs':(lambda filepath: ("srcml --xpath '//src:typedef' %s | xpath '//unit/typedef/descendant::text()'"%(filepath)),"XPATH"),
                        'Pointers':(lambda filepath: get_pointer_declarations(filepath),'MIXED'),
                        'References':(lambda filepath: get_reference_declarations(filepath),'MIXED'),
                        'Arrays':(lambda filepath: get_array_declarations(filepath),'MIXED'),
                        'Function_Calls':(lambda filepath:get_function_calls(filepath),'MIXED'),
                        'CodeLines':(lambda filepath: get_codelines(filepath),'FUNCTION')
                        #'Struct_definitions':(lambda filepath: get_struct_def(filepath),'MIXED')
                        }
    tree=ET.parse(filepath)
    root=tree.getroot()
    for feature_name,feature_l in feature_generators.items():
        feature_lambda,feature_lambda_type=feature_l
        generated_features=None
        if feature_lambda_type=='XPATH':
            generated_features=call_shell(feature_lambda,filepath)
        elif feature_lambda_type=='FUNCTION':
            generated_features=feature_lambda(root)
        else:
            generated_features=feature_lambda(filepath)
        features.append((feature_name,generated_features))
    return features
        
        
def generate_features(folder_path):
    df = defaultdict(lambda: [])
    c=0
    for f in os.listdir(folder_path):
        filename=f
        submitter=int(filename.split('_')[0])
        submission_id=filename.split('_')[1][:-4]
        Sum=summary[summary['ParticipantID']==submitter]
#         print(Sum.head())
#         print(Sum.shape)
#         print(submission_id)
#         print(submitter)
#         print(Sum.iloc[0]['ProblemC_id']==submission_id)
        if len(Sum)==0 or Sum.iloc[0]['ProblemC_id']!=submission_id:
            continue
            
        f=os.path.join(folder_path,f)
        if os.path.isfile(f):
            features=get_features_for_file(f)
            df['filename'].append(filename)
            for feature in features:
                df[feature[0]].append(feature[1])
        c+=1
        if (c%100)==0:
            print(c)
    return pd.DataFrame(df)

In [73]:
# x=generate_features('./Data/Contest1245/all_xml')
# x.head()
x.to_csv('primitive_features_1245.csv',index=False)

In [217]:
get_features_for_file('./Data/Contest1236/all_xml/29087676_62789478.xml')

[('Function_Return_type', b'long long\n'),
 ('Function_Params', b'long long a\n'),
 ('Function_Name', ['quickpow', 'main']),
 ('Typedefs', b''),
 ('Pointers', []),
 ('References', []),
 ('Arrays', []),
 ('Struct_definitions', [])]

In [139]:
import xml.etree.ElementTree as ET
tree = ET.parse('./Data/Contest1236/all_xml/29087676_62789478.xml')
root=tree.getroot()


{}
{}


In [187]:
cmd='''srcml --xpath "//src:decl[src:type/src:modifier[.='*']]" ./example.cpp | xpath "//unit/decl/descendant::text()"'''
cmd='''srcml --xpath "//src:decl[src:type/src:modifier[.='*']]" ./example.cpp '''

def get_leaf_tags(root,result=""):
    if len(root)==0:
        result=result+" "+root.text
        return result
    else:
        for child in root:
            result=get_leaf_tags(child,result)
        return result

proc=subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
result=[]
root=ET.fromstring(out)
for unit in root:
    result.append(get_leaf_tags(unit))
print("FINALLY",result)

FINALLY [' int * c', ' char * b']


In [225]:
get_function_calls('./example.cpp')

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<unit xmlns="http://www.srcML.org/srcML/src" revision="0.9.5">\n\n<unit xmlns:cpp="http://www.srcML.org/srcML/cpp" revision="0.9.5" language="C++" filename="example.cpp" item="1"><call><name>min</name><argument_list>(<argument><expr><name>b</name></expr></argument>,<argument><expr><name>c</name><operator>/</operator><literal type="number">2</literal></expr></argument>)</argument_list></call></unit>\n\n<unit xmlns:cpp="http://www.srcML.org/srcML/cpp" revision="0.9.5" language="C++" filename="example.cpp" item="2"><call><name>min</name><argument_list>(<argument><expr><name>a</name></expr></argument>,<argument><expr><name>b</name><operator>/</operator><literal type="number">2</literal></expr></argument>)</argument_list></call></unit>\n\n</unit>\n'


[' min b c / 2', ' min a b / 2']

In [47]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 957 entries, 0 to 999
Data columns (total 15 columns):
ParticipantID        957 non-null int64
Rank                 957 non-null int64
Country              957 non-null object
ProblemA_id          957 non-null object
ProblemA_memory      957 non-null object
ProblemA_time        957 non-null object
ProblemA_language    957 non-null object
ProblemB_id          957 non-null object
ProblemB_memory      957 non-null object
ProblemB_time        957 non-null object
ProblemB_language    957 non-null object
ProblemC_id          957 non-null object
ProblemC_memory      957 non-null object
ProblemC_time        957 non-null object
ProblemC_language    957 non-null object
dtypes: int64(2), object(13)
memory usage: 119.6+ KB


In [74]:
x.head()

,filename,Function_Return_type,Function_Params,Function_Name,Typedefs,Pointers,References,Arrays,Function_Calls
0,29618584_63999953.xml,b'signed\n',b'\n',[main],b'typedef long long ll;typedef long double ld;...,[],[],"[ int a N, b N]","[ ios_base :: sync_with_stdio 0, cin . tie 0,..."
1,29579543_64002824.xml,b'void\n',b'll& a\n',"[mul_mod, add_mod, solve, main]",b'typedef long long ll;',[],"[ ll & a, ll & a]","[ ll memo 100100, bool done 100100]","[ ll 1e9 + 7, add_mod memo i solve i + 1, so..."
2,29605117_64004827.xml,b'int\n',b'\n',[main],b'typedef unsigned long long int ull;typedef l...,[],[],[ ll f N],"[ ios_base :: sync_with_stdio false, s . leng..."
3,29571979_64025086.xml,b'void\n',b'int p\n',"[build, cmp, dis, main]",b'typedef long long ll;',[],[],"[ int x M, y M, t M, q N, ti N << 2, ll a...","[ build p << 1 l mid, build p << 1 | 1 mid + ..."
4,29609021_64003066.xml,b'int\n',b'int a\n',"[gcd, solve, main]",b'',[],[],[],"[ gcd b a % b, s . size (), ios_base :: sync..."


In [78]:
#generate xml for all files using srcml
for outer_d in os.listdir('./DataAnalysis/Contest1245/China'):
    outer_d=os.path.join('./DataAnalysis/Contest1245/China',outer_d) #Contest1,2,3,...
    if os.path.isdir(outer_d):
        #create dir for xml 
        xml_dir=os.path.join(outer_d,"all_xml")
        if not os.path.exists(xml_dir):
            os.makedirs(xml_dir) 
        for f in os.listdir(outer_dir):
            fname=f
            f=os.path.join(outer_d,f)
            if os.path.isfile(f):
                cmd="srcml %s -o %s"%(f,os.path.join(xml_dir,fname.replace('.cpp','.xml')))
                os.system(cmd)

FileNotFoundError: [Errno 2] No such file or directory: './DataAnalysis/Contest1245/China/hm/all'